In [3]:
# Normal Comment
# ! Very Important Errors or things which must be fixed or be in attention immediately
# TODO: Things remaining to do
# ? Questions
# * Some Messages or Notes
# ^ Important Notes, Messages or things which may need attention in future
# & Important Commented out Code
# ~ Purple

In [4]:

#^###############################  Storage_Optimization.ipynb  ####################################
# ^ Author: Sukhendu Sain
# ^ Description: Main file of codebase. Houses main code
# ^ Data: 23-Nov-2024
#^################################################################################

In [5]:
# Import Necessary Libraries, Utils, and Config Files
import utils
from config import *
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
# import importlib
# importlib.reload(utils)

# Data Import and Clean

In [6]:
#### Read FILE:: (AKINS FoMoCo_Piece_Sales_112222_YTD.xlsx) into Dataframe
# df_Akins = utils.read_excel(AKINS_FOMO_FILE_PATH)
# df_Akins['Part#'] = df_Akins['Part#'].apply(lambda a: "".join(str(a).split('-')))
# if print_df_after_import: utils.print_df(df_Akins, 200) # Print the Dataframe
# ~1-2secs

In [7]:
#### Read FILE:: (GPARTS Part Measures.xlsx) into Dataframe
df_Gparts = utils.read_excel(GPARTS_FILE_PATH)
if print_df_after_import: utils.print_df(df_Gparts) # Print the Dataframe
# ~50-60secs

In [8]:
#### Read FILE:: (Wholesale JAN_Oct_Parts_Ranking_Counter_Invoices_All_Brands.xlsx) into Dataframe
df_Wholesale = utils.read_excel(WHOLESALE_FILE_PATH)

# Clean the Wholesale Dataframe
df_Wholesale['Description'] = df_Wholesale['Description'].astype(str)
df_Wholesale = df_Wholesale.drop(columns=[col for col in df_Wholesale.columns if 'Unnamed' in col], inplace=False)
df_Wholesale = df_Wholesale[(df_Wholesale['Vendor'] == 'FOR')].reset_index()
df_Wholesale.loc[df_Wholesale['Description'].apply(lambda x: len(x.split("      ")) > 1), 'Avg. Cost'] = df_Wholesale['Description'].apply(lambda x: [i for i in x.strip().split("      ")][-1])
df_Wholesale.loc[df_Wholesale['Description'].apply(lambda x: len(x.split("      ")) > 1), 'Description'] = df_Wholesale['Description'].apply(lambda x: "     ".join([i for i in x.strip().split("      ")][:-1]))

if print_df_after_import: utils.print_df(df_Wholesale) # Print the Dataframe
# ~12-15secs

In [9]:
#### Read FILE:: (Service JAN_Oct_Parts_Ranking_ROs_All_Brands.xlsx) into Dataframe
df_Service = utils.read_excel(SERVICE_FILE_PATH)

# Clean the Service Dataframe
df_Service['Description'] = df_Service['Description'].astype(str)
df_Service = df_Service.drop(columns=[col for col in df_Service.columns if 'Unnamed' in col], inplace=False)
df_Service = df_Service[(df_Service['Vendor'] == 'FOR')].reset_index()
df_Service.loc[df_Service['Description'].apply(lambda x: len(x.split("      ")) > 1), 'Avg. Cost'] = df_Service['Description'].apply(lambda x: [i for i in x.strip().split("      ")][-1])
df_Service.loc[df_Service['Description'].apply(lambda x: len(x.split("      ")) > 1), 'Description'] = df_Service['Description'].apply(lambda x: "     ".join([i for i in x.strip().split("      ")][:-1]))
df_Service.loc[df_Service['Qty Sold'].apply(lambda x: len(str(x).split("      ")) > 1), 'Dollars Sold'] = df_Service['Qty Sold'].apply(lambda x: [i for i in str(x).strip().split("      ")][-1])
df_Service.loc[df_Service['Qty Sold'].apply(lambda x: len(str(x).split("      ")) > 1), 'Qty Sold'] = df_Service['Qty Sold'].apply(lambda x: "     ".join([i for i in str(x).strip().split("      ")][:-1]))

if print_df_after_import: utils.print_df(df_Service, 100) # Print the Dataframe
# ~5-6secs

In [10]:
#### Read FILE:: (Counter Pad) into Dataframe

# Data Processing & Calculation

### Make a Big Final Dataframe

In [11]:

# * It will have the Columns - 'Part Number', 'Part Desc.', 'Active', 'Sold (Pcs.)', '0Dimensions', 'Length/Depth', 'Width', 'Height', 'Zone', 'Storage Type', 'Sub Storage', 'Number of Storage needed'
# It will have all the rows with common part nos. from 3 Files, having Appropriate Sold Pcs. Values, and Dimensions

main_list = [] # Initialize the New List, which will hold all rows before turning into DF
gParts_PartNos = set(df_Gparts['Svc Part Number']) # Get a Set of all Part Nos. of GParts


# Wholesale
common_part_numbers = gParts_PartNos & set(df_Wholesale['Part Number'])
df_PreMerge = df_Gparts.loc[df_Gparts['Svc Part Number'].isin(common_part_numbers), ['Svc Part Number', 'Svc Part Number Description', 'Is Active?', 'Prod Att - Length', 'Prod Att- Width', 'Prod Att - Height']]
for pn, pddesc, ac, dp, wd, ht in zip(df_PreMerge['Svc Part Number'], df_PreMerge['Svc Part Number Description'], df_PreMerge['Is Active?'], df_PreMerge['Prod Att - Length'], df_PreMerge['Prod Att- Width'], df_PreMerge['Prod Att - Height']):
    main_list.append([pn, pddesc, "", "Wholesale", ac, df_Wholesale[df_Wholesale['Part Number'] == pn]['Sold'].values[0], 0, 0, False, dp, wd, ht, "", "", "", "", 0, "", ""])


# Service
common_part_numbers = gParts_PartNos & set(df_Service['* indicates a superseded part\nPart Number'])
df_PreMerge = df_Gparts.loc[df_Gparts['Svc Part Number'].isin(common_part_numbers), ['Svc Part Number', 'Svc Part Number Description', 'Is Active?', 'Prod Att - Length', 'Prod Att- Width', 'Prod Att - Height']]
for pn, pddesc, ac, dp, wd, ht in zip(df_PreMerge['Svc Part Number'], df_PreMerge['Svc Part Number Description'], df_PreMerge['Is Active?'], df_PreMerge['Prod Att - Length'], df_PreMerge['Prod Att- Width'], df_PreMerge['Prod Att - Height']):
    main_list.append([pn, pddesc, "", "Service", ac, 0, df_Service[df_Service['* indicates a superseded part\nPart Number'] == pn]['Qty Sold'].values[0], 0, False, dp, wd, ht, "", "", "", "", 0, "", ""])


# Create the Main Dataframe
df_Main = pd.DataFrame(main_list)
df_Main.columns = ['Part#', 'Part Desc.', 'Part Category', 'DataSource', 'Active', 'Wholesale Sold', 'Service Sold', "Total Sold", '0Dimensions', 'Depth', 'Width', 'Height', 'Zone', 'StorageType', 'SubStorage', 'Bin Type', 'OH Inventory', 'Num. Storage Required', "Bin Location"]

# Merge the Parts present in both Service and Wholesale (Duplicates)
gbParts = df_Main.groupby('Part#').count()[df_Main.groupby('Part#').count()['Part Desc.'] == 2].index.to_list()
for pn in gbParts:
    df_Main.loc[(df_Main['Part#'] == pn) & (df_Main['DataSource'] == "Wholesale"), "Service Sold"] = df_Main.loc[(df_Main['Part#'] == pn) & (df_Main['DataSource'] == "Service"), "Service Sold"].values[0]
    df_Main = df_Main[(df_Main['Part#'] != pn) | (df_Main['DataSource'] == "Wholesale")]
# Set 0Dimensions
df_Main.loc[(df_Main["Depth"] == 0) | (df_Main["Height"] == 0) | (df_Main["Width"] == 0), "0Dimensions"] = True
# Drop 0Dimensions Rows if drop0Dims
if drop0Dims: df_Main = df_Main[df_Main["0Dimensions"] == False]
# Set Total_Sold
df_Main["Total Sold"] = df_Main["Wholesale Sold"].astype(int) + df_Main["Service Sold"].astype(int)
# Sort by 'Total Sold'
df_Main = df_Main.sort_values('Total Sold', ascending=False)
# ^ Add Random Values for OH Inventory temporarily
df_Main["OH Inventory"] = np.random.choice(np.arange(20), size=len(df_Main), replace=True)
# Reset Index
df_Main = df_Main.reset_index(drop=True)

KeyboardInterrupt: 

In [ ]:

# * TESTING SECTION ---- Change some data Manually
df_Main.loc[df_Main["Part#"] == '6F2Z1A189A', ["Part Desc.","0Dimensions", "Depth", "Height", "Width", "OH Inventory"]] = ["6F2Z1A189A-TIRE",False, 28,28,3,100]
df_Main.loc[df_Main["Part#"] == '7L1Z1A189A', ["Part Desc.","0Dimensions", "Depth", "Height", "Width", "OH Inventory"]] = ["7L1Z1A189A-TIRE",False, 34,34,3.5,50]
df_Main.loc[df_Main["Part#"] == '9OO1183106436', ["0Dimensions", "Depth", "Height", "Width", "OH Inventory"]] = [False, 30,30,3,1000]
df_Main.loc[df_Main["Part#"] == '9OO439510', ["0Dimensions", "Depth", "Height", "Width", "OH Inventory"]] = [False, 45,45,4.5,500]
df_Main.loc[df_Main["Part#"] == '9OO1732002500', ["0Dimensions", "Depth", "Height", "Width", "OH Inventory"]] = [False, 50,50,5,250]
df_Main.loc[df_Main["Part#"] == '9OO3004901', ["0Dimensions", "Depth", "Height", "Width", "OH Inventory"]] = [False, 40,40,4,25]
df_Main[['TIRE' in s for s in df_Main["Part Desc."]]]

In [ ]:
utils.print_df(df_Main)

In [ ]:

###  ^ PART CATEGORIZATION. TO DO.  BUT Client Will Share Actual Part Category
def part_categorization(df_toBeCategorized):
    # TODO: Add more categories
    for i in range(df_toBeCategorized.shape[0]):
        desc = "-".join(df_toBeCategorized.loc[i, "Part Desc."].split("-")[1:])
        category = ""
        if "rivet" in desc.lower():
            category = "Rivet"
        elif ("blade" in desc.lower()) & ("wiper" in desc.lower()):
            category = "Wiper Blade"        
        elif ("arm" in desc.lower()) & ("wiper" in desc.lower()):
            category = "Wiper Arm"
        elif ("v-belt" in desc.lower()):
            category = "V-Belt"
        elif ("belt" in desc.lower()) & ("retractor" not in desc.lower()) & ("hole" not in desc.lower()) & ("cover" not in desc.lower()):
            category = "Belt"
        # elif ("hose" in desc.lower()) & ("vent" not in desc.lower()) & ("connect" not in desc.lower() & ("radiator" not in desc.lower()& ("heater" not in desc.lower()):
        #     category = "Hose"

        df_toBeCategorized.loc[i, 'Part Category'] = category

### Apply Zoning

In [ ]:

## * Main Function for Apply Zoning
def Apply_Zoning(df_toBeZoned, zones, soldColName='Total Sold', zoneColName='Zone'): 
    df_toBeZoned.loc[:, zoneColName] = df_toBeZoned[soldColName].apply(lambda x: next((zone for zone, ratio in zones.items() if x >= ratio), list(zones.keys())[0]))
    df_toBeZoned.loc[df_toBeZoned[soldColName] < 0, zoneColName] = None

In [ ]:
## Run the Apply_Zoning on df_Main
Apply_Zoning(df_Main, zones, 'Total Sold', 'Zone')

In [ ]:
## Check each Zone's number of Part Numbers
df_Main['Zone'].value_counts()

### Specialty Storage Assignment

#### Function for Bin Calculation

In [ ]:
def getNumOfBin(depth, width, height, raw_bin_dim, ohInven):
    # ^ Raw Bin Dimensions has this format :-  Height_Depth_Width
    FILL_FACTOR = 0.7       #  70% Filling, 30% Open Space for DRAWER / RACK / SHELVE

    if (raw_bin_dim != "") and (ohInven > 0):
        bin_height = float(raw_bin_dim.split("_")[1])
        bin_depth = float(raw_bin_dim.split("_")[2])
        bin_width = float(raw_bin_dim.split("_")[3])

        if raw_bin_dim.split("_")[0] == "BR":        ###  HANDLE BATTERY
            return round((bin_depth / width), 2)

        if raw_bin_dim.split("_")[0] == "TR":        ###  HANDLE TIRES 
            return 0
        
        if raw_bin_dim.split("_")[0] == "BC":        ###  HANDLE BUMPER COVERS  
            return 0
        if raw_bin_dim.split("_")[0] == "HS":        ###  HANDLE HANGING ITEMS 
            return 0
            


        # numOfBins = math.ceil(ohInven / (math.floor(bin_depth / depth) * math.floor(bin_width / width) * math.floor(bin_height / height)))
        volBin = FILL_FACTOR * bin_height * bin_depth * bin_width       ###  Available Storage Space
        volPart = height * depth * width
        numOfBins = round(ohInven / (volBin // volPart), 2)            ###  Returns Fraction. 

        return numOfBins
    else:
        return 0

### Main Function for Storage Assignment

In [ ]:
def getStorage(zone, pdesc, depth, width, height, ohInven):
    # Initialize the empty Variables
    storageType = ""
    subStorage = ""

    isSpec, storageType, subStorage, raw_bin_dim = utils.getSpecialityStorage(pdesc, depth, width, height)

    if not isSpec: 
        if (zone == "Red Hot") | (zone == "Red"):
            storageType, subStorage, raw_bin_dim = utils.getRedHotStorage(depth, width, height)
        elif zone == "Orange":
            storageType, subStorage, raw_bin_dim = utils.getOrangeStorage(depth, width, height)
        elif zone == "Yellow":
            storageType, subStorage, raw_bin_dim = utils.getYellowStorage(depth, width, height)
        elif zone == "Green":   
            storageType, subStorage, raw_bin_dim = utils.getGreenStorage(depth, width, height)
        elif zone == "Blue":
            storageType, subStorage, raw_bin_dim = utils.getBlueStorage(depth, width, height)

    numOfBins = getNumOfBin(depth, width, height, raw_bin_dim, ohInven)
    binDim =  raw_bin_dim.split('_')[0] + raw_bin_dim.split('_')[3] + raw_bin_dim.split('_')[2] + raw_bin_dim.split('_')[1]
    

    return storageType, subStorage, binDim, numOfBins # Return the Values

#### Apply the Storage Function

In [ ]:
for i in range(df_Main.shape[0]):
    # Set the Dimensions of the Data into Variables
    depth = df_Main.loc[i, "Depth"]
    height = df_Main.loc[i, "Height"]
    width = df_Main.loc[i, "Width"]

    zone = df_Main.loc[i, "Zone"]
    pdesc = df_Main.loc[i, "Part Desc."]
    ohInven = df_Main.loc[i, "OH Inventory"]

    # If any dimension is zero, set empty Storage
    if df_Main.loc[i, "0Dimensions"] == True:
        df_Main.loc[i, "StorageType"] = ""
        df_Main.loc[i, "SubStorage"] = ""
        continue

    # Set Storage of the Parts
    df_Main.loc[i, "StorageType"], df_Main.loc[i, "SubStorage"], df_Main.loc[i, "Bin Type"], df_Main.loc[i, "Num. Storage Required"] = getStorage(zone, pdesc, depth, width, height, ohInven)


In [ ]:

## ^ Tire Storage Calculation
# TODO: CLARIFY the Calculation of Tire Carousel Model Selection Based on Percentage 
## * ASSUMPTION:  If 50% of Tires Have 33-inch or More Diameter, Assign Large-Storage (72-width carousel)
## * ELSE,  For 28-inch or less, and,  between 28-33 inches, assign standard carrousel (48-width carousel) 
# & df_Main.loc[df_Main['StorageType'] == 'Tire Specialty Storage', 'SubStorage'] 
carousel_model = '72-inch Wide' if df_Main[df_Main['StorageType'] == 'Tire Specialty Storage'][df_Main['SubStorage'] == '>33-inches Wide'].shape[0] / df_Main[df_Main['StorageType'] == 'Tire Specialty Storage'].shape[0] >= TIRE_PERCENT1 else '48-inch Wide'
#carousel_width = '72' if df_Main[df_Main['StorageType'] == 'Tire Specialty Storage'][df_Main['SubStorage'] == '>33-inches Wide'].shape[0] / df_Main[df_Main['StorageType'] == 'Tire Specialty Storage'].shape[0] >= TIRE_PERCENT1 else '48'
for tirePN in df_Main.loc[df_Main['StorageType'] == 'Tire Specialty Storage', "Part#"]:
    df_Main.loc[(df_Main['Part#'] == tirePN), "Num. Storage Required"] = getNumOfBin(df_Main.loc[(df_Main['Part#'] == tirePN), "Depth"].values[0], df_Main.loc[(df_Main['Part#'] == tirePN), "Width"].values[0], df_Main.loc[(df_Main['Part#'] == tirePN), "Height"].values[0], f"{df_Main.loc[(df_Main['Part#'] == tirePN), "Height"].values[0]}_{df_Main.loc[(df_Main['Part#'] == tirePN), "Depth"].values[0]}_{carousel_width}", df_Main.loc[(df_Main['Part#'] == tirePN), "OH Inventory"].values[0])
    df_Main.loc[(df_Main['Part#'] == tirePN), "StorageType"] += carousel_model + " Carousel"
    

In [ ]:
utils.print_df(df_Main)

In [ ]:
df_Main.to_excel('FinalDataset.xlsx', index=False) 

In [ ]:
df_Main["StorageType"].value_counts()

In [ ]:
df_Main[(df_Main["0Dimensions"] == True) & (df_Main["StorageType"] == "")]